In [ ]:
import copy
import csv
import gc
import operator
import os
import pathlib
import shutil

import numpy as np
import PIL
import pydegensac
from scipy import spatial
import tensorflow as tf
from tqdm import tqdm
import tensorflow.keras.layers as L

import subprocess
subprocess.call(['pip', 'install', '../input/landmark-ckp/faixx/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl'])

In [ ]:
import faiss
import sys
package_path = '../input/efficientnet100minimal/'
package_path2 = '../input/keras-applications/'
sys.path.append(package_path)
sys.path.append(package_path2)
import keras_applications
import efficientnet.tfkeras as efn 

In [ ]:
INPUT_DIR = os.path.join('..', 'input')

DATASET_DIR = os.path.join(INPUT_DIR, 'landmark-recognition-2020')
TEST_IMAGE_DIR = os.path.join(DATASET_DIR, 'test')
TRAIN_IMAGE_DIR = os.path.join(DATASET_DIR, 'train')
TRAIN_LABELMAP_PATH = os.path.join(DATASET_DIR, 'train.csv')

NUM_PUBLIC_TRAIN_IMAGES = 1580470
MAX_NUM_EMBEDDINGS = -1

NUM_TO_RERANK = 4
TOP_K = 3

MAX_INLIER_SCORE = 30
MAX_REPROJECTION_ERROR = 4.0
MAX_RANSAC_ITERATIONS = 1000
HOMOGRAPHY_CONFIDENCE = 0.99

SAVED_MODEL_DIR = '../input/delg-saved-models/local_and_global'
DELG_MODEL = tf.saved_model.load(SAVED_MODEL_DIR)
DELG_IMAGE_SCALES_TENSOR = tf.convert_to_tensor([0.70710677, 1.0, 1.4142135])
DELG_SCORE_THRESHOLD_TENSOR = tf.constant(175.)
DELG_INPUT_TENSOR_NAMES = [
    'input_image:0', 'input_scales:0', 'input_abs_thres:0'
]

In [ ]:
NUM_EMBEDDING_DIMENSIONS = 2048
GLOBAL_FEATURE_EXTRACTION_FN = tf.saved_model.load('../input/google-landmark-subs')

In [ ]:
LOCAL_FEATURE_NUM_TENSOR = tf.constant(1000)
LOCAL_FEATURE_EXTRACTION_FN = DELG_MODEL.prune(
    DELG_INPUT_TENSOR_NAMES + ['input_max_feature_num:0'],
    ['boxes:0', 'features:0'])

LANDMARK_CLS = tf.keras.models.load_model('../input/landmark-ckp/efnb2_cls_/efnb2_v2.h5')

def to_hex(image_id) -> str:
  return '{0:0{1}x}'.format(image_id, 16)


def get_image_path(subset, image_id):
  name = to_hex(image_id)
  return os.path.join(DATASET_DIR, subset, name[0], name[1], name[2],
                      '{}.jpg'.format(name))


def load_image_tensor(image_path):
  return tf.convert_to_tensor(
      np.array(PIL.Image.open(image_path).convert('RGB')))


def extract_global_features(image_root_dir, n_splits = None, split = None):

  if n_splits:
    
      image_paths = [x for x in pathlib.Path(image_root_dir).rglob('*.jpg')]
    
      num_images = len(image_paths)
      split_size = len(image_paths) // n_splits  
      start = split - 1
      end = split
    
      image_paths = image_paths[(start * split_size): min(end * split_size, len(image_paths))]
    
  else:

      image_paths = [x for x in pathlib.Path(image_root_dir).rglob('*.jpg')]

  num_embeddings = len(image_paths)
  if MAX_NUM_EMBEDDINGS > 0:
    num_embeddings = min(MAX_NUM_EMBEDDINGS, num_embeddings)

  ids = num_embeddings * [None]
  embeddings = np.empty((num_embeddings, NUM_EMBEDDING_DIMENSIONS))

  for i, image_path in tqdm(enumerate(image_paths)):
    if i >= num_embeddings:
        break

    ids[i] = int(image_path.name.split('.')[0], 16)

    image_tensor = load_image_tensor(image_path)

    embedding_tensor = GLOBAL_FEATURE_EXTRACTION_FN.call(image_tensor)['global_descriptor']
    embeddings[i, :] = embedding_tensor.numpy()
    
  return ids, embeddings


def extract_local_features(image_path):

  image_tensor = load_image_tensor(image_path)

  features = LOCAL_FEATURE_EXTRACTION_FN(image_tensor, DELG_IMAGE_SCALES_TENSOR,
                                         DELG_SCORE_THRESHOLD_TENSOR,
                                         LOCAL_FEATURE_NUM_TENSOR)


  keypoints = tf.divide(
      tf.add(
          tf.gather(features[0], [0, 1], axis=1),
          tf.gather(features[0], [2, 3], axis=1)), 2.0).numpy()

  descriptors = tf.nn.l2_normalize(
      features[1], axis=1, name='l2_normalization').numpy()

  return keypoints, descriptors


def get_putative_matching_keypoints(test_keypoints,
                                    test_descriptors,
                                    train_keypoints,
                                    train_descriptors,
                                    max_distance=0.9):

  train_descriptor_tree = spatial.cKDTree(train_descriptors)
  _, matches = train_descriptor_tree.query(
      test_descriptors, distance_upper_bound=max_distance)

  test_kp_count = test_keypoints.shape[0]
  train_kp_count = train_keypoints.shape[0]

  test_matching_keypoints = np.array([
      test_keypoints[i,]
      for i in range(test_kp_count)
      if matches[i] != train_kp_count
  ])
  train_matching_keypoints = np.array([
      train_keypoints[matches[i],]
      for i in range(test_kp_count)
      if matches[i] != train_kp_count
  ])

  return test_matching_keypoints, train_matching_keypoints


def get_num_inliers(test_keypoints, test_descriptors, train_keypoints,
                    train_descriptors):

  test_match_kp, train_match_kp = get_putative_matching_keypoints(
      test_keypoints, test_descriptors, train_keypoints, train_descriptors)

  if test_match_kp.shape[
      0] <= 4:  
    return 0

  try:
    _, mask = pydegensac.findHomography(test_match_kp, train_match_kp,
                                        MAX_REPROJECTION_ERROR,
                                        HOMOGRAPHY_CONFIDENCE,
                                        MAX_RANSAC_ITERATIONS)
  except np.linalg.LinAlgError:  
    return 0

  return int(copy.deepcopy(mask).astype(np.float32).sum())


def get_total_score(num_inliers, global_score):
  local_score = min(num_inliers, MAX_INLIER_SCORE) / MAX_INLIER_SCORE
  return local_score + global_score


def rescore_and_rerank_by_num_inliers(test_image_id,
                                      train_ids_labels_and_scores):

  test_image_path = get_image_path('test', test_image_id)
  test_keypoints, test_descriptors = extract_local_features(test_image_path)

  for i in range(len(train_ids_labels_and_scores)):
    train_image_id, label, global_score = train_ids_labels_and_scores[i]

    train_image_path = get_image_path('train', train_image_id)
    train_keypoints, train_descriptors = extract_local_features(
        train_image_path)

    num_inliers = get_num_inliers(test_keypoints, test_descriptors,
                                  train_keypoints, train_descriptors)
    total_score = get_total_score(num_inliers, global_score)
    train_ids_labels_and_scores[i] = (train_image_id, label, total_score)

  train_ids_labels_and_scores.sort(key=lambda x: x[2], reverse=True)

  return train_ids_labels_and_scores


def load_labelmap():
  with open(TRAIN_LABELMAP_PATH, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    labelmap = {row['id']: row['landmark_id'] for row in csv_reader}

  return labelmap


def get_prediction_map(test_ids, train_ids_labels_and_scores):

  prediction_map = dict()

  for test_index, test_id in enumerate(test_ids):
    hex_test_id = to_hex(test_id)

    aggregate_scores = {}
    for _, label, score in train_ids_labels_and_scores[test_index][:TOP_K]:
      if label not in aggregate_scores:
        aggregate_scores[label] = 0
      aggregate_scores[label] += score

    label, score = max(aggregate_scores.items(), key=operator.itemgetter(1))

    prediction_map[hex_test_id] = {'score': score, 'class': label}

  return prediction_map


def decode_image(filename, label=None, image_size=(512, 512)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label
    
    
def landmark_cls_score(image_root_dir):
    image_paths = [x for x in pathlib.Path(image_root_dir).rglob('*.jpg')]
    image_paths = [os.fspath(x) for x in image_paths]
    
    
    
    landmark_cls_scores = []
    ids = []
    
    for p in image_paths:
        tmp = decode_image(p)
        score = LANDMARK_CLS.predict(tf.expand_dims(tmp, 0))
        landmark_cls_scores.append(np.float(score))
        
        ids.append(p.split('/')[-1].split('.')[0])
        
    return dict(zip(ids, landmark_cls_scores))


def get_predictions(labelmap):

  test_ids, test_embeddings = extract_global_features(TEST_IMAGE_DIR)

  print('faiss_1')

  train_ids, train_embeddings = extract_global_features(TRAIN_IMAGE_DIR, n_splits = 2, split = 1)

  train_ids = np.array([to_hex(x) for x in train_ids])

  faiss_index = faiss.IndexFlatIP(NUM_EMBEDDING_DIMENSIONS) 
  faiss_index.add(train_embeddings.astype('float32'))
  faiss_dist, faiss_index = faiss_index.search(test_embeddings.astype('float32'), NUM_TO_RERANK)

  lookup_ = np.vectorize(lambda x: labelmap[x])

  faiss_labels = lookup_(train_ids[faiss_index])

  combined1 = list(zip([[int(x, 16) for x in y] for y in train_ids[faiss_index].tolist()], 
                     faiss_labels.tolist(), 
                     faiss_dist.tolist()))
  train_ids_labels_and_scores = [list(zip(x[0], x[1], x[2])) for x in combined1]

  del train_ids
  del train_embeddings
  del faiss_index
  gc.collect()


  print('faiss_2')

  train_ids, train_embeddings = extract_global_features(TRAIN_IMAGE_DIR, n_splits = 2, split = 2)

  train_ids = np.array([to_hex(x) for x in train_ids])

  faiss_index = faiss.IndexFlatIP(NUM_EMBEDDING_DIMENSIONS) 
  faiss_index.add(train_embeddings.astype('float32'))
  faiss_dist, faiss_index = faiss_index.search(test_embeddings.astype('float32'), NUM_TO_RERANK)

  lookup_ = np.vectorize(lambda x: labelmap[x])

  faiss_labels = lookup_(train_ids[faiss_index])

  combined2 = list(zip([[int(x, 16) for x in y] for y in train_ids[faiss_index].tolist()], 
                     faiss_labels.tolist(), 
                     faiss_dist.tolist()))

  train_ids_labels_and_scores2 = [list(zip(x[0], x[1], x[2])) for x in combined2]
    
  train_ids_labels_and_scores = [c1 + c2  for c1, c2 in zip(train_ids_labels_and_scores, 
                                                           train_ids_labels_and_scores2)]
    
  train_ids_labels_and_scores = [sorted(x, key = lambda x: x[-1])[::-1][:NUM_TO_RERANK] for x in train_ids_labels_and_scores]

  del train_ids
  del train_embeddings
  del faiss_index
  gc.collect()
    
    
  pre_verification_predictions = get_prediction_map(
      test_ids, train_ids_labels_and_scores)

  for test_index, test_id in enumerate(test_ids):
    train_ids_labels_and_scores[test_index] = rescore_and_rerank_by_num_inliers(
        test_id, train_ids_labels_and_scores[test_index])

  post_verification_predictions = get_prediction_map(
      test_ids, train_ids_labels_and_scores)

  reg_cls_scores = landmark_cls_score(TEST_IMAGE_DIR)
  score = np.array([v for k, v in reg_cls_scores.items()])
    
  curr_avg = 0

  for threshold in [0.1, 0.01, 0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.000005, 0.000001, 0.0000005, 0.0000001][::-1]:
    
    curr_avg = np.mean(score < threshold)
    
    if curr_avg > 0.2:
        break
        

  for id_ in post_verification_predictions.keys():
    
    cls_score = reg_cls_scores[id_]
    
    if cls_score < threshold:
        post_verification_predictions[id_]['score'] = post_verification_predictions[id_]['score'] * cls_score
        
        
  from collections import Counter
  pred_classes = [v['class'] for k, v in post_verification_predictions.items()]
  pred_classes_counts = Counter(pred_classes)
  filtered_counts = Counter(el for el in pred_classes_counts.elements() if pred_classes_counts[el] >= 30)

  omit_ks = [k for k in filtered_counts]


  for id_ in post_verification_predictions.keys():

    lbl = post_verification_predictions[id_]['class']

    if lbl in omit_ks:
        post_verification_predictions[id_]['score'] = post_verification_predictions[id_]['score'] * -1
        
      
  return pre_verification_predictions, post_verification_predictions



def save_submission_csv(predictions=None):

  if predictions is None:

    shutil.copyfile(
        os.path.join(DATASET_DIR, 'sample_submission.csv'), 'submission.csv')
    return

  with open('submission.csv', 'w') as submission_csv:
    csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'landmarks'])
    csv_writer.writeheader()
    for image_id, prediction in predictions.items():
      label = prediction['class']
      score = prediction['score']
      csv_writer.writerow({'id': image_id, 'landmarks': f'{label} {score}'})

def main():
  labelmap = load_labelmap()
  num_training_images = len(labelmap.keys())
  print(f'Found {num_training_images} training images.')

  if num_training_images == NUM_PUBLIC_TRAIN_IMAGES:
    print('Found NUM_PUBLIC_TRAIN_IMAGES. Copying sample submission.')
    save_submission_csv()
    return

  _, post_verification_predictions = get_predictions(labelmap)
  save_submission_csv(post_verification_predictions)


if __name__ == '__main__':
  main()

In [ ]:
print("done!")